# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import google_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the csv generated in WeatherPy
weather_df = pd.read_csv('../WeatherPy/weather_check.csv')
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mana,5.66,-53.78,81.55,83,100,13.06,GF,1604030858
1,1,busselton,-33.65,115.33,88.00,21,0,2.53,AU,1604030859
2,2,carnarvon,-24.87,113.63,78.80,65,0,14.99,AU,1604030859
3,3,esperance,-33.87,121.90,73.40,28,93,13.87,AU,1604030859
4,4,punta arenas,-53.15,-70.92,46.40,70,75,12.75,CL,1604030859


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure out gmaps figure with our api key
gmaps.configure(api_key=google_api_key)

In [4]:
# Get the columns Lat and Lng for our location values
locations = weather_df[['Lat','Lng']].astype(float)
# Get humidty for our weight on the heatmap
humidity = weather_df['Humidity'].astype(float)

# Create the figure, create the heatmap layer
# Adjust the max intensity to be the humidity list
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = max(humidity), point_radius = 3, dissipating = False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Set the parameters for ideal weather (we're rather picky)
idealtemp = [65,75]
idealhumidity = 60
idealwind = 7
idealclouds = 50
# Remove the temperatures that are too hot or too cold
ideal_weather_df = weather_df.loc[(weather_df['Max Temp']<idealtemp[1]) & (weather_df['Max Temp']>idealtemp[0]),:]

# Remove the places with too high of humidity (sorry southeast US)
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Humidity'] <= idealhumidity),:]

# Remove winds that are too high
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed'] <= idealwind),:]

# Remove places that are too cloudy
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Cloudiness'] <= idealclouds),:]

#Check how many places we still have left after the culling
#Turns out >10 means losing points so must be as strict as possible.
ideal_weather_df.shape

(13, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Rename our dataframe and then create an empty list to hold hotels
hotel_df = ideal_weather_df
hotel_name = []

# Loop through the rows of the hotel dataframe and search for hotels within 5000m
for index,row in hotel_df.iterrows():
    
    params = {
        "location": "{0},{1}".format(row['Lat'],row['Lng']),
        "keyword": "hotel",
        "radius": 5000,
        "type": "lodging",
        "key": google_api_key
    }
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params = params).json()
    
    # Try to put the hotel name into our list
    try:
        hotel_name.append(response['results'][0]['name'])
    # If we can't find a hotel let the user know which city is lacking and put a blank in the hotel field
    except IndexError:
        print('Unfortunately, the city ' + str(row['City']) + 'has no hotels within 5000m')
        hotel_name.append('')
hotel_df['Hotel Name'] = hotel_name

Unfortunately, the city barahas no hotels within 5000m
Unfortunately, the city bafoulabehas no hotels within 5000m
Unfortunately, the city barkhanhas no hotels within 5000m


In [7]:
# Print the hotel list to see which ones we found and how may blanks we have
# Likely our blanks are from our multiple levels of abstraction. We have random positions on the globe and the nearest city.
# Then we searched for hotels within 5000m of the random locations which may itself be >5000m from the nearest town.
hotel_name

['Mweebe Executive Lodge',
 '',
 'Sofitel Legend Old Cataract Aswan',
 'Diplomat Hotel',
 'Shiv Parivar Resort',
 'castle de white house hotel kaduna',
 'Holiday Inn Mauritius Mon Tresor',
 'Laguna Beach Hotel & Spa',
 'LUX Grand Gaube Resort & Villas',
 'SkylineInn Hotel',
 '',
 '',
 'Best Western Plus Main Street Inn']

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))